Este Notebook contém minhas anotações, e é baseado no original disponível em: https://www.kaggle.com/code/phelpsmemo/self-organizing-maps 

## Self-Organizing Maps (SOM)

<img src="https://static.packt-cdn.com/products/9781788830577/graphics/7ced975d-4ffa-401c-b42b-1c5f85a99aa4.png" align="right" width="380">


Este documento apresenta um pequeno resumo do funcionamento da rede Self-Organizing Maps (SOM), também chamados de Mapas de Kohonen. Neste material serão apresentados os temas:

- **Aprendizado competitivo**
- **Processo de aprendizado da rede SOM**
- **Formas de avaliar o SOM**

> Este material foi criado dentro do contexto da disciplina do Instituto Nacional de Pesquisas Espaciais CAP-351, então é assumido que você tenha em mente os conceitos apresentados antes desta rede (Procesos de aprendizado, redes *Multi-Layer Perceptron*).


### Aprendizado competitivo

Antes de entrar no tópico central deste documento, é necessário entender um pouco sobre o conceito da aprendizagem competitiva, já que está é utilizada pelo SOM em seu processo de aprendizagem. Para começar, se lembrarmos como uma rede *Multi-Layer Perceptron* realiza seu processo de treinamento, teremos que há um processo de atualização sistemática nos pesos de **todos** os neurônios, que é feito através do algoritmo de retropropagação. 

Diferente desta lógico, no aprendizado competitivo os neurônios estão competindo para "ganhar o direito" de ativar quando um determinado padrão é apresentado para os neurônios. Esse direito normalmente vem através de uma função de distância entre o neurônio e o padrão que está sendo apresentado. 

Uma vez que o neurônio ganha o direito de ser ativo, ele é atualizado seguindo uma determinada regra de atualização. Note que, a diferença está justamente nesta etapa de atualização, aqui, somente o vencedor vai ser atualizado. É partindo desta ideia que o SOM realiza seu processo de aprendizado, que será apresentado abaixo.

### Processo de aprendizado da rede SOM

Dada uma amostra X do espaço de entrada representando um padrão de ativação aplicado à rede, três processos estarão envolvidos na formação do mapa auto-organizável.

**Competição**

Para cada padrão de entrada, os neurônios computam seus respectivos valores de uma função discriminante. Esta função disciminante fornece a base para a competição entre os neurônios. O neurônio que obtém o melhor resultado via função discriminante é declarado o neurônio vencedor (*Best matching unit* - BMU).

> A **função discriminante** é utilizada para calcular a **distância** entre o padrão de entrada e os pesos de cada um dos neurônios. Pode-se utilizar qualquer função de distância. Algumas possíveis são: **euclidiana**, **cosine**, **manhattan** e **chebyshev**.


Como forma de representação, então temos que:

- Vetor de entrada: x = $[x_1, x_2, ..., x_m]^T$
- Rede (Pesos dos neurônios): $w_j = [w_{j1}, w_{j2}, ..., w_{j3}]^T$, $j = 1, 2, ..., l$
- Neurônio vencedor: $i(x) = argmin ||x - w_j||$, $j = 1, 2, ..., l$

> Veja que o **argmin** está representando a menor distância, mas não há problemas em alterar esta definição se a função discriminante exigir uma lógica diferente.

**Cooperação**

O neurônio vencedor (BMU) determina a localização espacial de uma vizinhança (No grid da camada de saída) topológica de neurônios excitados, fornecendo assim a base para a cooperação entre neurônios vizinhos.

> Esta etapa é o que faz o SOM ser um pouco diferente de algoritmos que seguem a linha do aprendizado competivivo. Aqui, a lógica do "vencedor leva tudo", onde somente quem vence é atualizado não é utilizada. Com esta etapa de cooperação, a partir do momento que um neurônio ganhou a competicação e a capacidade de representar um determinado padrão ele "leva" seus vizinhos juntos para o processo de atualização, de modo que, os que estão mais próximos ao neurônio vencedor são mais beneficiados (Entenda o benefício como "ganhar o direito de ser atualizado" em uma determinada amplitude de valores) do que os que estão mais distante. 

> Note que, quando é dito sobre a "atualização do neurônio", está sendo indicado que este, terá seus pesos atualizados para que, ele fique "mais similar" ao padrão que foi apresentado. Isso faz com que os neurônios vão sendo especializados em entradas de um determinado formato. Além disso, quando os vizinhos são atualizados, é dito que, aqueles que estão próximos, vão acabam representando elementos similares, e por isso, eles também devem ser atualizados.


De maneira geral temos que nesta etapa:

- O neurônio vencedor $i(x)$, está no centro de uma região (vizinhança);
- Neurônios vizinhos se beneficiam da ativação do neurônio vencedor;
- Definição da vizinhança topológica do neurônio $i$ (Determina o grau de benefício dos neurônios vizinhos com base em sua distância do neurônio vencedor)

$h_{j , i(x)} = exp(-\frac{d_{j, i}^2}{2\sigma^2})$

> $d_{j, i}^2$ é a distância entre os neurônios no grid

- Decaimento da vizinhança (Representa a abertura da vizinhança. Quanto maior, mais generoso é o neurônio vencedor para seus vizinhos)

$\sigma(n) = \sigma_0 exp(-\frac{n}{\tau})$

> Para o entendimento, imagine que o sigma represente a abertura da gaussiana que é utilizada para representar a taxa de atualização dos neurônios vizinhos ao vencedor.

> Valores de sigma muito grande vão fazer com que básicamente o $h_{j, i(x)}$ tenda ao infinito, o que faz a atualização, para todos os neurônios ser 1. Ou seja, a distância do vencedor não importa, todos vão ser atualizados da mesma forma. A ideia é ir reduzindo esse sigma de modo que, no início, muitos neurônios sejam arrastados e com o passar do tempo, apenas o neurônio vencedor seja movido. Isso permite um "ajuste fino" nos neurônios da rede.


Com a definição desses parâmetros realizada, cabe dizer que a ideia da etapa de cooperação, de maneira geral, é a de permitir que dado um conjunto de entradas, o grid seja ajustado de maneira uniforme sob o conjunto de dados, assim, caso os parâmetros sejam configurados de maneira incorreta, isso acabará resultando em um ajuste de grid que não tem essa distribuição uniforme, o que causa problemas aos resultados gerados pela rede.

**Adaptação sináptica**

Este último mecanismo permite aos neurônios excitados ajustar seus valores individuais da função discriminante em relação ao padrão de entrada (Sua similaridade com o o padrão de entrada) através de modificações aplicadas a seus pesos sinápticos. Este ajuste é feito de tal maneira que a resposta do neurônio vencedor à uma subsequente aplicação de um padrão similar é ampliada, e pode ser alcançada atravás da formula:

$w_j(t + 1) = w_j(t) + \eta(t) \dot h_{j, i(x)}(t) \dot (x - w_j(t))$

onde,

$\eta(t) = \eta_0 exp(-\frac{t}{\tau_2})$ (Decaimento de $\eta$)

Note que a atualização dos pesos do neurônio, é ponderada pelo valor de $h_{j, i(x)}$

> Esta etapa representa o que anteriormente foi definido como a "movimentação do neurônio" para próximo do padrão apresentado, o que torna ele especializado no determinado padrão que está sendo apresentado. 

Com isso, após repetidas apresentações de dados de treinamento, os pesos sinápticos tendem a seguir a distribuição do vetor de entrada devido à esta atualização. O algoritmo portanto leva a uma ordenação topológica do mapa de características em relação ao espaço de entrada, no sentido que os neurônios adjacentes terão vetores de pesos similares.

**Ideia por trás dos decaimentos**: Se olharmos para o processo apresentado, temos que durante o processo de treinamento, a rede SOM tem duas funções de decaimento. Por trás da definição de cada uma delas, pode ser utilizada a análogia de "arrumar o lençol". Quando começamos a arrumar, os valores estão mais altos, logo vamos arrumando de qualquer jeito de modo a cobrir toda a área de cama. Depois que o lençol já está sob a cama, vamos fazendo ajustes menores, de modo a tratar dos problemas de cada região.

### Formas de avaliar o SOM

Para interpretar os resultados gerados no treinamento, é necessário que avaliações sejam realizadas. Nesta seção são apresentadas métricas para esta avaliação.


**Erro de quantização**: Quanto menor o erro de quantização, maior o indicativo de que, o neurônio que está vencendo e representando um certo padrão, está próximo a ele.

> Como forma de interpretação, aqui pode ser utilizada a seguinte ideia. "Não é porque você é o vencedor que realmente merece ser". Isso já que, ele pode estar ganhando por ser o mais próximo, ok, mas isso não indica que ele está "colado" à amostra. Imagine um *outlayer*, algum neurônio ganha o direito de representar ele, mas a distância euclidiana é 100, quando o normal, para esse nosso processo de treinamento fictício, é algo em torno de 0.5~0.7

$QE(M) = \frac{1}{n}\sum_{i = 1}^n||\phi(x_i) - x_i||$

Com relação a este erro, deve-se ter cuidado com relação a quantidade de neurônios. Se eu quiser erro zero, posso simplesmente colocar um neurônio por exemplo, o que possivelmente não vai representar o problema corretamente. No caminho inverso, se uma quantidade muito pequena de neurônios for utilizada, pode ser que o erro de quantização seja alto, mas ele representa bem o problema (Seguindo o critério de um especialista, por exemplo).

**Erro topográfico**: Utilizado para verificar a relação entre o neurônio vencedor e outros possíveis vencedores. A ideia aqui é que, dado um neurônio $N_1$, que venceu, este, deve ser vizinho de grid a um possível segundo neurônio vencedor $N_2$. Isso pode ser útil já que, a premissa é que, elementos que fazem representações de um mesmo elemento estejam próximos no grid.

> De maneira geral, a ideia da geração de redes com boas representações é a reduzir esse erro.

**U-Matrix**: Representa a soma das distâncias de um neurônio em relação aos seus vizinhos. A ideia é verificar a homogeneidade das regiões geradas no mapa.
